In [ ]:
from myai.imports import *
from visualbench.tasks import BoxPacking, FunctionDescent, MatrixInverse
from visualbench import Benchmark
import math
opt = lambda params, lr: sig(torch.optim.SGD, params, lr)



class Task:
    def __init__(self, name, new_fn, max_steps, max_time, max_passes, min_loss = None, test_every_sec = None): # TODO add max passes
        self.name = name
        self.new_fn = new_fn
        self.max_steps = max_steps
        self.max_time = max_time
        self.max_passes = max_passes # TODO pass
        self.min_loss = min_loss
        self.test_every_sec = test_every_sec

    def run(self, opt_fn, **kwargs):
        bench: Benchmark = self.new_fn()
        opt = opt_fn(bench.parameters(), **kwargs)
        bench.run(opt, max_steps=self.max_steps, max_time=self.max_time, min_loss=self.min_loss, test_every_sec=self.test_every_sec)

        return bench

tasks = [
    Task('rosen', lambda: FunctionDescent('rosen'), 1000, 3, 0.01),
    Task('booth', lambda: FunctionDescent('booth'), 1000, 3, 0.01),
    Task('matrix inverse', lambda: MatrixInverse(torch.randn(128,128)), 1000, 10, 0.01)]

lrs_log10 = [1, 0, -1, -2, -3, -4]
lr_binary_search_iters = 5

results = {}
class Result:
    def __init__(self, bench:Benchmark, task:Task):
        self.train_losses = bench.logger.get_metric_as_numpy('train loss')
        self.test_losses: dict[int, float] | None = bench.logger['test loss'] if 'test loss' in bench.logger else None

        self.num_passes = bench.current_step + bench.num_backwards # TODO num passes
        self.time_passed = bench.time_passed
        self.lowest_loss = bench.lowest_loss
        
        self.task = task


for t in tasks:
    results[t.name] = {}
    for lr in lrs_log10:
        bench = t.run(opt, lr = 10 ** lr)
        results[t.name][lr] = Result(bench, t)

    # extra lr eval
    for _ in range(lr_binary_search_iters):
        lr_losses = [(lr, res.lowest_loss) for lr, res in results[t.name].items()]
        lr_losses.sort(key = lambda x: x[1])
        sorted_lrs = sorted(results[t.name].keys())

        lr1 = lr_losses[0][0]; lr2 = lr_losses[1][0]
        if lr1 == sorted_lrs[0]: new_lr = lr1 - 1
        elif lr1 == sorted_lrs[-1]: new_lr = lr1 + 1
        else: new_lr = (lr1 + (lr2 - lr1) / 2)

        if round(new_lr, 3) not in lrs_log10:
            bench = t.run(opt, lr = 10 ** new_lr)
            results[t.name][new_lr] = Result(bench, t)
        else:
            break



In [28]:
{lr: t.lowest_loss for lr, t in results['matrix inverse'].items()}

{1: tensor(17.0688, grad_fn=<AddBackward0>),
 0: tensor(2.9078, grad_fn=<AddBackward0>),
 -1: tensor(15.5849, grad_fn=<AddBackward0>),
 -2: tensor(23.6448, grad_fn=<AddBackward0>),
 -3: tensor(34.1690, grad_fn=<AddBackward0>),
 -4: tensor(38.2560, grad_fn=<AddBackward0>),
 -0.5: tensor(10.7459, grad_fn=<AddBackward0>),
 -0.25: tensor(6.9472, grad_fn=<AddBackward0>),
 -0.125: tensor(4.7401, grad_fn=<AddBackward0>)}

In [29]:
10 ** -0.125

0.7498942093324559